In [ ]:
. ../../../../scripts/nbs_header.ps1
. ../../../../scripts/core.ps1

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --build-file test.spi test.fsx --timeout 10000 } | Invoke-Block

00:00:00 #1 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #2 [Debug] executeAsync / options: { Command =
   "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:main@415-297> }
00:00:00 #3 [Verbose] > pwd: /home/runner/work/polyglot/polyglot/apps/spiral/temp/test
00:00:00 #4 [Verbose] > dll_path: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:00 #5 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #6 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:00 #7 [Verbose] testPortOpen / ex: System.AggregateException:

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.13.0: F# to Rust compiler (status: alpha)

Thanks to the contributor! @irium
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
.> dotnet restore test.fable-temp.csproj -p:FABLE_COMPILER=true -p:FABLE_COMPILER_4=true -p:FABLE_COMPILER_RUST=true
  Determining projects to restore...
  Paket version 8.0.0-alpha002+6339fce88f07b4d959d03b78330fdb5a6d936483
  The last full restore is still up to date. Nothing left to do.
  Total time taken: 0 milliseconds
  Paket version 8.0.0-alpha002+6339fce88f07b4d959d03b78330fdb5a6d936483
  Restoring /home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.fable-temp.csproj
  Starting restore process.
  Total time taken: 0 milliseconds
  Restored /home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.fable-temp.csproj (in 252 ms).
.> dotnet restore /home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.fsproj
  Determining projects to restore...
  Restored /home/runner/work/polyglot/polyglot/apps/sp

In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
| Set-Content test.rs

In [ ]:
cargo fmt --

In [ ]:
cargo build --release

   Compiling fable_library_rust v0.1.0 (/home/runner/work/polyglot/polyglot/apps/spiral/temp/test/fable_modules/fable-library-rust)
   Compiling spiral_temp_test v0.0.1 (/home/runner/work/polyglot/polyglot/apps/spiral/temp/test)
  --> apps/spiral/temp/test/./main.rs:11:27
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                           ^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `any` is already imported here
   |
   = note: `#[warn(unused_imports)]` on by default

  --> apps/spiral/temp/test/./main.rs:11:32
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                                ^^^^^^^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `Arbitrary` is already imported here

   --> apps/spiral/temp/test/./main.rs:190:15
    |
190 | use tracing::{info, Level};
    |               ^^^^

  --> apps/spiral/temp/test/./main.rs:46:8
   |
45 | impl Cart {
   | --------- associated items in this implementatio

In [ ]:
$env:RUST_LOG="info"
{ cargo test --release } | Invoke-Block

   Compiling fable_library_rust v0.1.0 (/home/runner/work/polyglot/polyglot/apps/spiral/temp/test/fable_modules/fable-library-rust)
   Compiling spiral_temp_test v0.0.1 (/home/runner/work/polyglot/polyglot/apps/spiral/temp/test)
  --> apps/spiral/temp/test/./main.rs:11:27
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                           ^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `any` is already imported here
   |
   = note: `#[warn(unused_imports)]` on by default

  --> apps/spiral/temp/test/./main.rs:11:32
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                                ^^^^^^^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `Arbitrary` is already imported here

    Finished `release` profile [optimized] target(s) in 13.00s
     Running unittests main.rs (/home/runner/work/polyglot/polyglot/target/release/deps/spiral_temp_test-a65dbe0c6620615a)

running 3 tests
test test_parse_number ... 

In [ ]:
{ . $ScriptDir/../../../../target/release/spiral_temp_test$(GetExecutableSuffix) } | Invoke-Block

app=test
